# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [26]:
import logging
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
import sklearn.preprocessing
from sklearn.metrics.pairwise import cosine_similarity
import sklearn.metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
import scipy
from implicit.als import AlternatingLeastSquares

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [2]:
# !wget https://storage.yandexcloud.net/mle-data/ym/tracks.parquet

# !wget https://storage.yandexcloud.net/mle-data/ym/catalog_names.parquet

# !wget https://storage.yandexcloud.net/mle-data/ym/interactions.parquet

In [3]:
tracks = pd.read_parquet('tracks.parquet')
catalog_names = pd.read_parquet('catalog_names.parquet')
interactions = pd.read_parquet('interactions.parquet')


# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [4]:
tracks.head()

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]
3,136,"[12, 214, 2490809]",[84],[11]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 24...",[84],[11]


In [5]:
catalog_names.head()

,id,type,name
0,3,album,Taller Children
1,12,album,Wild Young Hearts
2,13,album,Lonesome Crow
3,17,album,Graffiti Soul
4,26,album,Blues Six Pack


In [6]:
interactions.head()

,user_id,track_id,track_seq,started_at
0,0,99262,1,2022-07-17
1,0,589498,2,2022-07-19
2,0,590262,3,2022-07-21
3,0,590303,4,2022-07-22
4,0,590692,5,2022-07-22


In [7]:
print('Количество пропущенных значений в tracks:')
print(tracks.isna().sum())

print('\n\nКоличество пропущенных значений в catalog_names:')
print(catalog_names.isna().sum())

print('\n\nКоличество пропущенных значений в interactions:')
print(interactions.isna().sum())


Количество пропущенных значений в tracks:
track_id    0
albums      0
artists     0
genres      0
dtype: int64


Количество пропущенных значений в catalog_names:
id      0
type    0
name    0
dtype: int64


Количество пропущенных значений в interactions:
user_id       0
track_id      0
track_seq     0
started_at    0
dtype: int64


In [8]:
print('Типы данных в tracks:')
print(tracks.dtypes)

print('\n\nТипы данных в catalog_names:')
print(catalog_names.dtypes)

print('\n\nТипы данных в interactions:')
print(interactions.dtypes)

Типы данных в tracks:
track_id     int64
albums      object
artists     object
genres      object
dtype: object


Типы данных в catalog_names:
id       int64
type    object
name    object
dtype: object


Типы данных в interactions:
user_id                int32
track_id               int32
track_seq              int16
started_at    datetime64[ns]
dtype: object


In [9]:
print('Количество уникальных пользователей в interactions:')
print(interactions['user_id'].nunique())

print('Количество уникальных треков в interactions:')
print(interactions['track_id'].nunique())

print('\n\nКоличество уникальных треков в tracks:')
print(tracks['track_id'].nunique())

print('\n\nКоличество уникальных объектов в catalog_names:')
print(catalog_names['id'].nunique())

Количество уникальных пользователей в interactions:
1373221
Количество уникальных треков в interactions:
1000000


Количество уникальных треков в tracks:
1000000


Количество уникальных объектов в catalog_names:
1776697


In [10]:
print('Минимальная дата в interactions:')
print(interactions['started_at'].min())

print('\n\nМаксимальная дата в interactions:')
print(interactions['started_at'].max())

Минимальная дата в interactions:
2022-01-01 00:00:00


Максимальная дата в interactions:
2022-12-31 00:00:00


In [11]:
# проверка на наличие треков в interactions, которых нет в tracks
interactions[~interactions['track_id'].isin(tracks['track_id'].unique())]

,user_id,track_id,track_seq,started_at


In [12]:
all_albums = set(np.concatenate(tracks['albums'].tolist()))
albums_without_names = all_albums - set(catalog_names['id'].tolist())
print(albums_without_names)

set()


In [13]:
all_genres = set(np.concatenate(tracks['genres'].tolist()))
genres_without_names = all_genres - set(catalog_names['id'].tolist())
print(genres_without_names)

{160, 161, 130, 131, 164, 168, 169, 153, 155, 124, 159}


In [14]:
all_artists = set(np.concatenate(tracks['artists'].tolist()))
artists_without_names = all_artists - set(catalog_names['id'].tolist())
print(artists_without_names)

set()


In [15]:
all_tracks = set(tracks['track_id'].unique())
tracks_without_names = all_tracks - set(catalog_names['id'].tolist())
print(tracks_without_names)

set()


In [16]:
tracks_without_genres = tracks[tracks['genres'].apply(lambda x: any(genre in genres_without_names for genre in x))]['track_id'].unique()



In [17]:
interactions[interactions['track_id'].isin(tracks_without_genres)]['track_id'].shape[0] / interactions['track_id'].shape[0]

0.021482743526208686

In [18]:
len(tracks_without_genres)

18323

In [19]:
catalog_names['type'].value_counts(normalize=True)

type
track     0.551733
album     0.363440
artist    0.084736
genre     0.000092
Name: proportion, dtype: float64

In [20]:
catalog_names.head(5)

,id,type,name
0,3,album,Taller Children
1,12,album,Wild Young Hearts
2,13,album,Lonesome Crow
3,17,album,Graffiti Soul
4,26,album,Blues Six Pack


In [21]:
genres_without_names_df = pd.DataFrame(genres_without_names, columns=['id'])
genres_without_names_df['type'] = 'genre'
genres_without_names_df['name'] = 'unknown genre'
catalog_names = pd.concat([genres_without_names_df, catalog_names], ignore_index=True)

In [23]:
default_album = 0
default_genre = 12345
default_artist = 1

tracks_albums = tracks[['track_id', 'albums']].explode('albums')
tracks_albums['albums'].fillna(default_album, inplace=True)
tracks_albums['albums'] = tracks_albums['albums'].astype(int)

tracks_genres = tracks[['track_id', 'genres']].explode('genres')
tracks_genres['genres'].fillna(default_genre, inplace=True)
tracks_genres['genres'] = tracks_genres['genres'].astype(int)

tracks_artists = tracks[['track_id', 'artists']].explode('artists')
tracks_artists['artists'].fillna(default_artist, inplace=True)
tracks_artists['artists'] = tracks_artists['artists'].astype(int)


print(tracks.shape)
print(tracks_albums.shape)
print(tracks_genres.shape)
print(tracks_artists.shape)

(1000000, 4)
(3128826, 2)
(1656345, 2)
(1279581, 2)


In [24]:
events = interactions.copy()

In [25]:
del interactions

# Выводы

Приведём выводы по первому знакомству с данными:
- В данных были выявлены две проблемы:  
    a) жанры некоторых треков без названия  
    b) id жанров, альбомов и артистов в виде списка, в котором могут быть пропуски и сложности просчета  
    c) атрибут started_at в неудобном формате numpy.datetime

- Были произведены следующие действия:  
    a) жанры без названия добавлены с название "unknown genre"  
    b) id жанров, альбомов и артистов представлены в виде отдельных таблиц с присвоением типа int каждому id  
    c) переводим started_at в тип pd.datetime

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

In [28]:
user_stats = events.groupby('track_id').agg(tracks_amt = ('user_id', 'nunique'))
print('Минимальное количество пользователей, прослушавших трек:', user_stats['tracks_amt'].min())
print('Максимальное количество пользователей, прослушавших трек:', user_stats['tracks_amt'].max())
print('Среднее количество пользователей, прослушавших трек:', user_stats['tracks_amt'].mean())
print('Медиана количества пользователей, прослушавших трек:', user_stats['tracks_amt'].median())
print('Квартиль 0.05 количества пользователей, прослушавших трек:', user_stats['tracks_amt'].quantile(0.05))
print('Квартиль 0.25 количества пользователей, прослушавших трек:', user_stats['tracks_amt'].quantile(0.25))
print('Квартиль 0.75 количества пользователей, прослушавших трек:', user_stats['tracks_amt'].quantile(0.75))
print('Квартиль 0.95 количества пользователей, прослушавших трек:', user_stats['tracks_amt'].quantile(0.95))

Минимальное количество пользователей, прослушавших трек: 5
Максимальное количество пользователей, прослушавших трек: 111062
Среднее количество пользователей, прослушавших трек: 222.629898
Медиана количества пользователей, прослушавших трек: 19.0
Квартиль 0.05 количества пользователей, прослушавших трек: 5.0
Квартиль 0.25 количества пользователей, прослушавших трек: 9.0
Квартиль 0.75 количества пользователей, прослушавших трек: 67.0
Квартиль 0.95 количества пользователей, прослушавших трек: 760.0


Наиболее популярные треки

In [32]:
catalog_names.set_index('id', inplace=True)

In [59]:
print('Топ-10 популярных треков:\n')

for track_id in user_stats.sort_values(by='tracks_amt', ascending=False).head(10).index:
    print(catalog_names.loc[track_id, 'name'])

Топ-10 популярных треков:

Smells Like Teen Spirit
Believer
Numb
I Got Love
Юность
Way Down We Go
Shape of You
In The End
Shape Of My Heart
Life


Наиболее популярные жанры

In [60]:
genre_stats = user_stats.reset_index().merge(tracks_genres, on='track_id', how='left')
genre_stats = genre_stats.groupby('genres').agg(tracks_amt = ('tracks_amt', 'sum'))
genre_stats.sort_values(by='tracks_amt', ascending=False, inplace=True)

print('Наиболее популярные жанры:\n')
genre_names = catalog_names.query("type == 'genre'")

for genre_id in genre_stats.sort_values(by='tracks_amt', ascending=False).head(10).index:
    print(genre_names.loc[genre_id, 'name'])

Наиболее популярные жанры:

pop
rap
allrock
ruspop
rusrap
electronics
dance
rusrock
rock
metal


Треки, которые никто не прослушал

In [58]:
len(set(tracks['track_id']))

1000000

In [57]:
len(set(events['track_id']))

1000000

In [54]:
set(tracks['track_id']) - set(events['track_id'])

set()

Нет треков, которые не были прослушаны, таблица tracks включает только прослушанные треки

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

In [68]:
events.head(5)

,user_id,track_id,track_seq,started_at
0,0,99262,1,2022-07-17
1,0,589498,2,2022-07-19
2,0,590262,3,2022-07-21
3,0,590303,4,2022-07-22
4,0,590692,5,2022-07-22


# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

In [75]:
import os
from dotenv import load_dotenv
import boto3
import s3fs

load_dotenv()

S3_BUCKET_NAME = os.getenv('S3_BUCKET_NAME')
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

bucket_path = 'recsys/data/'

In [80]:
fs = s3fs.S3FileSystem(
    key=AWS_ACCESS_KEY_ID,
    secret=AWS_SECRET_ACCESS_KEY
)

# Сохраняем DataFrame в S3
tracks.to_parquet(f'{S3_BUCKET_NAME}/{bucket_path}/items.parquet', filesystem=fs)

PermissionError: The AWS Access Key Id you provided does not exist in our records.

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

# Топ популярных

Рассчитаем рекомендации как топ популярных.

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.